In [1]:
import numpy as np
from molecular_mpns.config import data_dir
from molecular_mpns.proto_molecule import Molecule
from molecular_mpns.math import km_estimators
from molecular_mpns.math import PerronFrobenius2D,galerkin_coords
from d3s.domain import discretization
from d3s.observables import periodic_gaussians,gaussians
from d3s.algorithms import gedmd

In [2]:
# set up molecular environment 
N = 5
beta = 15.0
kb = 1.0
rb = 2.0
ka = 1.0
ra = np.pi/2
kd = np.array([[0.02, 3], [0.02, 2]])
mol = Molecule(N, beta, kb, rb, ka, ra, kd)

# gedmd parameters
bounds,boxes = np.array([[-2.5,2.5],[-2.5,2.5]]),np.array([10,10]) # 2.5,2.4
omega = discretization(bounds,boxes)
sigma = np.sqrt(0.05)
psi = gaussians(Omega = omega,sigma = sigma)

# load data
X,Y,Z = np.load(str(data_dir)+'/proto_angles.npy') , np.load(str(data_dir)+'/proto_effective_drift.npy'), np.load(str(data_dir)+'/proto_effective_diffusion.npy')    

In [3]:
# reference gedmd

np.random.seed(42)
nsamps = 2000000

rand_idx = np.random.choice(X.shape[1],nsamps)
X_samp,Y_samp,Z_samp = X[:,rand_idx],Y[:,rand_idx],Z[:,:,rand_idx]
A,d,V = gedmd(X_samp,-Y_samp,Z_samp,psi,diffusion = True,evs=20)

In [67]:
# km gedmd
dt = 5e-3
Y_km,Z_km =  np.zeros((X.shape[1],2)),np.zeros((2,2,X.shape[1]))

for i in range(X.shape[1]-1):
    x_current,x_next = X.T[i],X.T[i+1]
    y,z = km_estimators(x_current,x_next,dt)
    Y_km[i,:] = y
    Z_km[:,:,i] = z

np.random.seed(42)
rand_idx = np.random.choice(X.shape[1],nsamps)
X_samp,Y_samp,Z_samp = X[:,rand_idx],Y_km[rand_idx,:],Z_km[:,:,rand_idx]
A_km,d_km,V_km = gedmd(X_samp,Y_samp.T,Z_samp,psi = psi, diffusion = False,evs = 20)

In [4]:
# reference MSM model
angles = np.load(str(data_dir)+'/proto_xi_traj.npy')
angles = angles.T
pf = PerronFrobenius2D(15,np.array([[-np.pi,np.pi],[-np.pi,np.pi]]))
pf.estimate(angles,normalize = True)
ts = pf.implied_timescales(dt = 5e-3,n_vals = 20)
print(ts)

(array([1.        , 0.99575226, 0.99471611, 0.99388649, 0.990332  ,
       0.98993615, 0.97834235, 0.97298399, 0.97142238, 0.96705227,
       0.96204326, 0.96135165, 0.95948184, 0.9564025 , 0.95482047,
       0.95413367, 0.95319133, 0.9529866 , 0.95128978, 0.94944094]), array([-5.00000028e+04,  1.17462364e+00,  9.43787691e-01,  8.15372086e-01,
        5.14671233e-01,  4.94328439e-01,  2.28357257e-01,  1.82564694e-01,
        1.72450597e-01,  1.49242003e-01,  1.29213136e-01,  1.26855525e-01,
        1.20884532e-01,  1.12167156e-01,  1.08150569e-01,  1.06493104e-01,
        1.04298083e-01,  1.03832812e-01,  1.00127263e-01,  9.63728191e-02]))
